<a href="https://colab.research.google.com/github/luanakwon/GreatBarrierReef/blob/main/GBR_f2score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download dataset from kaggle api
kaggle  
https://www.kaggle.com/c/tensorflow-great-barrier-reef/data  

kaggle API usage  
https://colab.research.google.com/github/corrieann/kaggle/blob/master/kaggle_api_in_colab.ipynb

In [1]:
!pip install kaggle
!pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 58 kB 4.6 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=bc875cb6bd5724c56c35046c7ab078f19256b6dc34e73968ed995d91602f5147
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 68 bytes


In [ ]:
!kaggle competitions download -c tensorflow-great-barrier-reef

 44% 6.19G/14.2G [00:53<00:39, 215MB/s]

In [ ]:
!unzip -q /content/tensorflow-great-barrier-reef.zip 

# Organize dataset

In [ ]:
# get train.csv
import pandas as pd

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [ ]:
# create hierarchy
import os

for f2 in 'images', 'labels':
  os.makedirs(f'/content/Datasets/val/{f2}')

In [ ]:
def annotation2YoloFormat(_annotation, width=1280, height=720):
  out= ''
  bboxes = eval(_annotation)
  for x in bboxes:
    x = list(x.values())
    out += '0 %.6f %.6f %.6f %.6f\n'%(
      max(0,min(1,(x[0]+x[2]/2)/width)),
      max(0,min(1,(x[1]+x[3]/2)/height)),
      max(0,min(1,x[2]/width)),
      max(0,min(1,x[3]/height))
    )
  return out.lstrip()

In [ ]:
import random
import shutil

VAL_DIRECTORY = "/content/Datasets/val"

random.seed(10)

df = df_train
for index, row in df.iterrows():
  path_from = f'/content/train_images/video_{row.video_id}/{row.video_frame}.jpg'
  path_to = os.path.join(VAL_DIRECTORY,f'images/{row.image_id}.jpg')
  lb_path_to = os.path.join(VAL_DIRECTORY,f'labels/{row.image_id}.txt')

  random_value = random.random()
  if random_value < .64:
    # if row.annotations != '[]':
    #   shutil.move(row.image_path, f'train/images/{row.image_id}.jpg')
    #   with open(f'train/labels/{row.image_id}.txt', 'w') as f:
    #     for i in row.bbox:
    #       f.write("0 {} {} {} {}\n".format(*i))
    continue
  else:
    shutil.move(path_from, path_to)
    if row.annotations != '[]':
      with open(lb_path_to, 'w') as f:
        f.write(annotation2YoloFormat(row.annotations))

# Inference on Validation dataset

In [ ]:
import torch
import cv2
import numpy as np

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

from yolov5 import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v6.0-248-gcb2ad9f torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 75.0/166.8 GB disk)


In [ ]:
def load_model(ckpt_path, conf=0.25, iou=0.50):
  model = torch.hub.load('/content/yolov5',
                          'custom',
                          path=ckpt_path,
                          source='local',
                          force_reload=True)  # local repo
  model.conf = conf  # NMS confidence threshold
  model.iou  = iou  # NMS IoU threshold
  model.classes = None   # (optional list) filter by class, i.e. = [0, 15, 16] for persons, cats and dogs
  model.multi_label = False  # NMS multiple labels per box
  model.max_det = 1000  # maximum number of detections per image
  return model

In [ ]:
def predict(model, img, size=768, augment=False):
  height, width = img.shape[:2]
  results = model(img, size=size, augment=augment)  # custom inference size
  preds   = results.pandas().xyxy[0]
  bboxes  = preds[['xmin','ymin','xmax','ymax']].values
  if len(bboxes):
    bboxes[:,2:] -= bboxes[:,:2]
    bboxes  = bboxes.astype(int)
    confs   = preds.confidence.values
    return bboxes, confs
  else:
    return [],[]

def format_prediction(bboxes, confs):
    annot = ''
    if len(bboxes)>0:
        for idx in range(len(bboxes)):
            xmin, ymin, w, h = bboxes[idx]
            conf             = confs[idx]
            annot += f'{conf} {xmin} {ymin} {w} {h}'
            annot +=' '
        annot = annot.strip(' ')
    return annot

In [ ]:
# calculate IoU
# pred = (x, y, width, height)
# label = (x, y, width, height)
def getIoU(pred, label):

  area_label = label[2] * label[3]
  area_predict = pred[2] * pred[3]
  area_of_overlap = max(0,min(label[0]+label[2]*0.5, pred[0]+pred[2]*0.5)\
                  - max(label[0]-label[2]*0.5, pred[0]-pred[2]*0.5))\
                  * \
                  max(0,min(label[1]+label[3]*0.5, pred[1]+pred[3]*0.5) \
                  - max(label[1]-label[3]*0.5, pred[1]-pred[3]*0.5))
  area_of_union = area_label + area_predict - area_of_overlap
  return area_of_overlap / area_of_union

def getConfusion(bboxes, confis, labels, iou_thres,
                 conf_min=0, conf_max=1, conf_step=0.1):
  # list of [x,y,w,h,conf]
  b4c1 = [[*b,c] for b, c in zip(bboxes,confis)]
  b4c1.sort(key=lambda x : x[4]) # sort in ascending order
  # confusion matrix [conf_thres,tp,fp,fn]
  cm = np.zeros((int((conf_max-conf_min)/conf_step),4))
  for i, c in enumerate(cm):
    c[0] = conf_min+i*conf_step
  # make confusion matrix for all conf thres
  for conf_idx in range(cm.shape[0]):
    lbs = labels.copy()
    bc = b4c1.copy()
    for i, x in enumerate(bc):
      if x[4] > cm[conf_idx,0]:
        bc = bc[i:]
        break
    
    trues = len(lbs)
    # find pred-label match
    for sub_box in reversed(bc):
      for l in lbs:
        if getIoU(sub_box[:-1],l) > iou_thres:
          cm[conf_idx,1] += 1
          bc.remove(sub_box)
          lbs.remove(l)
          break
    # remains are False Positive and False Negative each
    cm[conf_idx,2] = len(bc)
    cm[conf_idx,3] = len(lbs)
  
    
  return cm

In [ ]:
IMG_SIZE = 2048
CONF_MIN = 0.25
CONF_MAX = 0.8
CONF_STEP = 0.1
VAL_DIRECTORY = '/content/Datasets/val'
LEN_VAL_DIR = len(os.listdir(f'{VAL_DIRECTORY}/images'))
conf_steps = int((CONF_MAX-CONF_MIN)/CONF_STEP)

my_model = load_model('/content/best.pt', conf=CONF_MIN, iou=0.3)
m_confusions = np.zeros((11,conf_steps,4)) 

for idx,img_f in enumerate(os.listdir(f'{VAL_DIRECTORY}/images')):
  if idx > 100:
    break
  
  path = f'{VAL_DIRECTORY}/images/{img_f}'
  print(f'\r[{idx}/{LEN_VAL_DIR}]{path}',end='\t\t')
  img = cv2.imread(path)[...,::-1]
  annot = eval(df_train[df_train.image_id == img_f[:-4]].annotations.item())
  # my model
  bboxes, confis = predict(my_model, img, size=2048, augment=True)

  for iou_idx in range(11): # iou thres from 0.3 to 0.8 step 0.05
    iou_thres = 0.05*iou_idx+0.3
    labels = [list(x.values()) for x in annot]
    m_confusions[iou_idx] += getConfusion(bboxes, confis, labels, iou_thres,
                                          CONF_MIN,CONF_MAX,CONF_STEP)

       

print()
# result - my model
confusions = m_confusions
print(confusions)
# prevent zero division
no_tp = confusions[:,:,1] == 0
confusions[:,:,1] += no_tp

precisions = confusions[:,:,1]/(confusions[:,:,1]+confusions[:,:,2])
recalls = confusions[:,:,1]/(confusions[:,:,1]+confusions[:,:,3])
f2s = (5*precisions*recalls)/(4*precisions+recalls)

f2s *= np.logical_not(no_tp)
print(f2s)
print(np.mean(f2s,axis=(0,2)))